# Process Rosbag and plot

In [ ]:
# Auto Reload Modules
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rc, font_manager, transforms

from bag_utils import extract_traj
from plot_utils import plot_map_2d, plot_map_3d, plot_car_frames

np.random.seed(0)

In [ ]:
test_x = np.linspace(0, 6, num=100)
test_y = np.sin(test_x)

W = 0.265
L = 0.52

fig = plt.figure()
ax = plt.gca()

ax.plot(test_x, test_y, linewidth=2)

blue_car = mpimg.imread('car_icons_blue.png')

pos_list = [np.array([0, 0, 0]), np.array([np.pi/2, 1, np.pi/2])]

plot_car_frames(ax, pos_list, blue_car, plot_shape=[W, L])

plt.xlim([0, 6])
plt.ylim([-1.5, 1.5])


## Load bag and extract data

In [ ]:
bag_files = glob.glob("./bag_data/*.bag")
print("Found bags:", bag_files)

In [ ]:
bag_num = 0

FSM, EV, TV = extract_traj(bag_files[bag_num])

## Visualization

In [ ]:
# Generate occupancy randomly
occupancy = np.random.rand(2, 19)
occupancy[occupancy <= 0.5] = 0
occupancy[occupancy > 0.5] = 1
# Set the destination of target vehicle to be empty
occupancy[0, 10] = 0

# Plot the figure in an interactive window
%matplotlib qt

rc('font', **{'family': "Times New Roman"})

### Plot 2D trajectoies

In [ ]:
fig_name = "ParkingLot2D"
x_boundary = [-0.1, 6.1]
y_boundary = [-1.5, 1.5]

fig = plt.figure(fig_name, figsize=(6.5, 2.5))

plot_map_2d(fig_name, occupancy=occupancy)

plt.plot(TV.x, TV.y, linewidth=4, color='g', label="TV")
plt.plot(EV.x, EV.y, linewidth=4, color='b', label="EV")

plt.legend(loc=(0.13, 0.65), prop={'size': 10})

plt.axis("equal")
plt.xlim(x_boundary)
plt.ylim(y_boundary)

ax = plt.gca()

W = 0.265
L = 0.52

blue_car = mpimg.imread('car_icons_blue.png')
green_car = mpimg.imread('car_icons_green.png')

pos_idxs = [int(i/5*len(EV.x)) for i in range(5)]

pos_list = [np.array([EV.x[idx], EV.y[idx], EV.heading[idx]]) for idx in pos_idxs]

plot_car_frames(ax, pos_list, blue_car, plot_shape=[W, L])

plt.xlim([0, 6])
plt.ylim([-1.5, 1.5])

ax.set_axis_off()

fig.tight_layout()

### Plot 3D trajectories (+Time)

In [ ]:
fig_name = "ParkingLot3D"

fig = plt.figure(fig_name, figsize=(15, 15), dpi=200)

# plot_map_3d(fig_name, occupancy=occupancy)
plot_map_3d(fig_name, height=EV.state_t[-1]-EV.state_t[0], occupancy=occupancy)

ax = plt.gca()

ax.plot3D(TV.x, TV.y, TV.state_t, linewidth=4, color='g', label="TV")
ax.plot3D(EV.x, EV.y, EV.state_t, linewidth=4, color='b', label="EV")

ax.set_yticks(ticks=[-1, 0, 1])
ax.set_zticks(ticks=[0, 5, 10, 15, 20])
# ax.tick_params(labelsize=10)

ax.set_xlabel("X (m)", labelpad=10, fontdict={"fontname": "Times New Roman", "fontsize": 15, "fontweight": "bold"})
ax.set_ylabel("Y (m)", fontdict={"fontname": "Times New Roman", "fontsize": 15, "fontweight": "bold"})
ax.set_zlabel("Time (s)", fontdict={"fontname": "Times New Roman", "fontsize": 15, "fontweight": "bold"})

font = font_manager.FontProperties(family="Times New Roman")

ax.legend(loc=(0.85,0.7), prop=font)

ax.set_box_aspect([2.5,1,1])

fig.tight_layout()

### Plot Speed

In [ ]:
fig_name = "Speed"
fig = plt.figure(fig_name, figsize=(12, 5))
plt.plot(EV.state_t, EV.v, linewidth=2, color='b', label="EV")

plt.xlabel("Time (s)", fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
plt.ylabel("Speed (m/s)", fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
plt.yticks(font="Times New Roman", size=15)
plt.xticks(font="Times New Roman", size=15)

fig.tight_layout()

### Plot Input

In [ ]:
fig_name = "Input"
fig = plt.figure(fig_name, figsize=(12, 5))

ax1 = plt.gca()
ax1.plot(EV.input_t, EV.delta, linewidth=4, color='C1', label="Steering")

ax1.set_xlabel("Time (s)", fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})

ax1.set_ylabel("Steering (rad)", color='C1', fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
ax1.tick_params(axis='y', labelcolor='C1', labelsize=15)

ax2 = ax1.twinx()

ax2.plot(EV.input_t, EV.a, linewidth=4, color='C2', label="Acceleration")

ax2.set_ylabel("Acceleration (m/s$^2$)", color='C2', fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
ax2.tick_params(axis='y', labelcolor='C2', labelsize=15)

fig.tight_layout()

### Plot Scores

In [ ]:
fig_name = "Score"
fig = plt.figure(fig_name, figsize=(12, 5))
plt.plot(FSM.t, FSM.score_l, linewidth=4, color='C1', label="Left")
plt.plot(FSM.t, FSM.score_r, linewidth=4, color='C2', label="Right")
plt.plot(FSM.t, FSM.score_y, linewidth=4, color='C3', label="Yield")

plt.legend(prop={'size': 20})

plt.xlabel("Time (s)", fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
plt.ylabel("Confidence", fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
plt.xticks(size=15)
plt.yticks(size=15)
plt.grid(axis='y')

fig.tight_layout()

### Plot Mode

In [ ]:
fig_name = "FSM"
fig = plt.figure(fig_name, figsize=(12, 5))
plt.plot(FSM.t, FSM.state_idxs, linewidth=4, color='C1')

plt.xlabel("Time (s)", fontdict={"fontname": "Times New Roman", "fontsize": 20, "fontweight": "bold"})
plt.yticks(ticks=np.arange(7), labels=FSM.state_names, font="Times New Roman", size=15)
plt.xticks(font="Times New Roman", size=15)
plt.grid(axis='y')

fig.tight_layout()